# Applied Data Science Capstone Project

## Gregory Smith

The body of this notebook consists of completing the requirements of the Applied Data Science capstone project as part of the Applied Data Science specialization on Coursera.

In [2]:
import pandas as pd
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
